In [1]:
#import key packages
import pandas as pd
import numpy as np
import time
import requests
from datetime import datetime, timedelta
import random
pd.set_option('mode.chained_assignment',None)

In [2]:
#create the dictionaries that will map each team acronym onto the same format
class MissingDict(dict):
    __missing__ = lambda self, key: key
acronyms = {
'Arizona' : 'ARI', 'L.A. Dodgers' : 'LAD', 'N.Y. Mets' : 'NYM', 'N.Y. Yankees' : 'NYY', 'Tampa Bay' :  'TBR', 
'Oakland' : 'OAK', 'Baltimore' : 'BAL', 'St. Louis' : 'STL', 'Kansas City' : 'KCR', 
'Milwaukee' : 'MIL', 'Toronto' : 'TOR', 'L.A. Angels' : 'LAA', 'Boston' : 'BOS', 'Seattle' : 'SEA', 
'Pittsburgh' : 'PIT', 'Miami' : 'MIA', 'Cleveland' : 'CLE', 'Texas' : 'TEX', 'Atlanta' : 'ATL', 'Chi. Cubs' : 'CHC', 
'Chi. White Sox' : 'CHW', 'Detroit' : 'DET', 'Minnesota' : 'MIN', 'Cincinnati' : 'CIN', 'Philadelphia' : 'PHI', 
'Washington' : 'WSN', 'San Francisco' : 'SFG', 'San Diego' : 'SDP', 'Colorado' : 'COL', 'Houston' : 'HOU'
}
maps = MissingDict(**acronyms)

class MissingDict(dict):
    __missing__ = lambda self, key: key
conversions = {
'Arizona Diamondbacks' : 'ARI', 'Los Angeles Dodgers' : 'LAD', 'New York Mets' : 'NYM', 'New York Yankees' : 'NYY', 'Tampa Bay Rays' :  'TBR', 
'Oakland Athletics' : 'OAK', 'Baltimore Orioles' : 'BAL', 'St. Louis Cardinals' : 'STL', 'Kansas City Royals' : 'KCR', 
'Milwaukee Brewers' : 'MIL', 'Toronto Blue Jays' : 'TOR', 'Los Angeles Angels' : 'LAA', 'Boston Red Sox' : 'BOS', 'Seattle Mariners' : 'SEA', 
'Pittsburgh Pirates' : 'PIT', 'Miami Marlins' : 'MIA', 'Cleveland Indians' : 'CLE', 'Cleveland Guardians' : 'CLE', 'Texas Rangers' : 'TEX', 'Atlanta Braves' : 'ATL', 'Chicago Cubs' : 'CHC', 
'Chicago White Sox' : 'CHW', 'Detroit Tigers' : 'DET', 'Minnesota Twins' : 'MIN', 'Cincinnati Reds' : 'CIN', 'Philadelphia Phillies' : 'PHI', 
'Washington Nationals' : 'WSN', 'San Francisco Giants' : 'SFG', 'San Diego Padres' : 'SDP', 'Colorado Rockies' : 'COL', 'Houston Astros' : 'HOU',
'Florida Marlins' : 'MIA', 'Montreal Expos' : 'WAS', 'Anaheim Angels' : 'LAA', 'Tampa Bay Devil Rays' : 'TB'
}
mapping = MissingDict(**conversions)

class MissingDict(dict):
    __missing__ = lambda self, key: key
winner_cons = {
'ARI' : 'ARI', 'LAD' : 'LAD', 'NYM' : 'NYM', 'NYY' : 'NYY', 'TB' :  'TBR', 
'OAK' : 'OAK', 'BAL' : 'BAL', 'STL' : 'STL', 'KC' : 'KCR', 
'MIL' : 'MIL', 'TOR' : 'TOR', 'LAA' : 'LAA', 'BOS' : 'BOS', 'SEA' : 'SEA', 
'PIT' : 'PIT', 'MIA' : 'MIA', 'CLE' : 'CLE', 'CLE' : 'CLE', 'TEX' : 'TEX', 'ATL' : 'ATL', 'CHC' : 'CHC', 
'CHW' : 'CHW', 'DET' : 'DET', 'MIN' : 'MIN', 'CIN' : 'CIN', 'PHI' : 'PHI', 
'WAS' : 'WSN', 'SF' : 'SFG', 'SD' : 'SDP', 'COL' : 'COL', 'HOU' : 'HOU',
'MIA' : 'MIA', 'LAA' : 'LAA', 'TB' : 'TB'
}
mapping = MissingDict(**winner_cons)

In [3]:
#create the parameters used for the pitcher specific data
useful = ['Home Away', 'Home Home', 'Home Winner', 'Home Loser','Home Name','Home W', 'Home L', 'Home SV',
       'Home G', 'Home GS', 'Home IP', 'Home K/9', 'Home BB/9', 'Home HR/9',
       'Home BABIP', 'Home LOB%', 'Home GB%', 'Home HR/FB', 'Home vFA (pi)',
       'Home ERA', 'Home xERA', 'Home FIP', 'Home xFIP', 'Home WAR', 'Away Name','Away W', 'Away L', 'Away SV',
       'Away G', 'Away GS', 'Away IP', 'Away K/9', 'Away BB/9', 'Away HR/9',
       'Away BABIP', 'Away LOB%', 'Away GB%', 'Away HR/FB', 'Away vFA (pi)',
       'Away ERA', 'Away xERA', 'Away FIP', 'Away xFIP', 'Away WAR','Home Run Line']

In [4]:
# Define the start and end day used for each season
start_mon,start_day,end_mon,end_day = 3,31,9,30

In [5]:
# Define the years included in the data
years = [x for x in range(2013,2023) if x != 2020]

In [6]:
all_stuff = []
tt = 0
for year in years:
    print(year)
    start_date = f'{year}-03-31'
    all_matches = []
    bat_reg =[]
    bat_adv = []
    pit_reg = []
    pit_adv = []
    sp_v = []
    # iterate through all dates from March 30th to September 30th inclusive
    for date1 in np.arange(datetime(year,start_mon,start_day), datetime(year,end_mon,end_day), timedelta(days=1)).astype(datetime):
        tt += 1
        time.sleep(.2)
        date = date1.strftime("%Y-%m-%d")
        try:

            # acquire the schedule for the given day from cbs
            cbs_url = f"https://www.cbssports.com/mlb/schedule/{date.replace('-','')}/"
            matches = pd.read_html(requests.get(cbs_url).content)[0]
            matches['Away'] = matches['Away'].map(acronyms)
            matches['Home'] = matches['Home'].map(acronyms)
            matches['Winner'] = np.empty(len(matches.index))
            matches['Loser'] = np.empty(len(matches.index))
            matches.insert(1,"Date",[date] * len(matches.index))
            for j in matches.index:
                matches.loc[j,'Winner'] = str(matches[matches.index == j]['Result']).split('    ')[1].split(' ')[0]
                matches.loc[j,'Loser'] = str(matches[matches.index == j]['Result']).split('    ')[1].split(' ')[3]
            all_matches.append(matches)
            time.sleep(.2)
            # get the regular hitting stats from fangraphs
            fg_url = f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=0&season={year}a&month=1000&season1={year}&ind=0&team=0%2Cts&rost=0&age=0&filter=&players=0&startdate={start_date}&enddate={date}'
            c = pd.read_html(requests.get(fg_url).content)[-2].iloc[:30,:]
            c.columns = c.columns.droplevel()
            c.insert(1,'Date',[date] * 30)
            bat_reg.append(c)
            time.sleep(.2)
            # get the advanced hitting stats from fangraphs 
            fg_url = f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=1&season={year}&month=1000&season1={year}&ind=0&team=0,ts&rost=0&age=0&filter=&players=0&startdate={start_date}&enddate={date}'
            c = pd.read_html(requests.get(fg_url).content)[-2].iloc[:30,:]
            c.columns = c.columns.droplevel()
            c.insert(1,'Date',[date] * 30)
            bat_adv.append(c)
            time.sleep(.2)
            # get the regular pitching stats from fangraphs
            fg_url = f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=0&season={year}&month=1000&season1={year}&ind=0&team=0,ts&rost=0&age=0&filter=&players=0&startdate={start_date}&enddate={date}'
            c = pd.read_html(requests.get(fg_url).content)[-2].iloc[:30,:]
            c.columns = c.columns.droplevel()
            c.insert(1,'Date',[date] * 30)
            pit_reg.append(c)
            time.sleep(.2)
            # get the advanced pitching stats from fangraphs
            fg_url = f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=1&season={year}&month=1000&season1={year}&ind=0&team=0%2Cts&rost=0&age=0&filter=&players=0&startdate={start_date}&enddate={date}'
            c = pd.read_html(requests.get(fg_url).content)[-2].iloc[:30,:]
            c.columns = c.columns.droplevel()
            c.insert(1,'Date',[date] * 30)
            pit_adv.append(c)
            if not tt % 10:
                print(date)
        except ValueError or KeyError:
            continue
    #combine the stats for a given year
    bat_reg1 = pd.concat(bat_reg)
    bat_adv1 = pd.concat(bat_adv)
    pit_reg1 = pd.concat(pit_reg)
    pit_adv1 = pd.concat(pit_adv)

    pit_reg1.drop(columns = '#',inplace=True)
    pit_adv1.drop(columns = '#',inplace= True)
    p_cols = [x for x in pit_reg1.columns if x in pit_adv1.columns]

    pit_stats = pd.merge(pit_reg1,pit_adv1,on = p_cols)
    # merge the pitching stats
    bat_reg1.drop(columns = '#',inplace=True)
    bat_adv1.drop(columns = '#',inplace= True)

    b_cols = [x for x in bat_reg1.columns if x in bat_adv1.columns]
    # merge the hitting stats
    hit_stats = pd.merge(bat_reg1,bat_adv1,on = b_cols)
    # filter out games without a result (postponed games),the result is denoted by the score in the format a-b
    # where a is the winning team's score and b is the losing team's score
    df = pd.concat(all_matches)
    df = df[df.Result.str.count('-') > 0]

    # create dataframes with the hitting and pitching stats
    hit_df = df.merge(hit_stats.add_prefix('Home_Hit_'),left_on = ['Date','Home'],right_on = ['Home_Hit_Date','Home_Hit_Team']).merge(hit_stats.add_prefix('Away_Hit_'),left_on = ['Date','Away'],right_on = ['Away_Hit_Date','Away_Hit_Team'])
    pit_df =  df.merge(pit_stats.add_prefix('Home_Pit_'),left_on = ['Date','Home'],right_on = ['Home_Pit_Date','Home_Pit_Team']).merge(pit_stats.add_prefix('Away_Pit_'),left_on = ['Date','Away'],right_on = ['Away_Pit_Date','Away_Pit_Team'])

    # merge them together and drop all duplicates
    final_df =pd.merge(hit_df,pit_df,on = [x for x in hit_df.columns if x in pit_df.columns],how = 'inner').drop_duplicates(subset = [x for x in hit_df.columns if x in pit_df.columns])
    final_df = final_df.drop_duplicates(subset = ['Home','Away','Date'])
    final_df = final_df[final_df.Result.str.count('-') > 0]
    all_stuff.append(final_df)

2013
2013-04-09
2013-04-19
2013-04-29
2013-05-09
2013-05-19
2013-05-29
2013-06-08
2013-06-18
2013-06-28
2013-07-08
2013-07-28
2013-08-07
2013-08-17
2013-08-27
2013-09-06
2013-09-16
2013-09-26
2014
2014-04-06
2014-04-16
2014-04-26
2014-05-06
2014-05-16
2014-05-26
2014-06-05
2014-06-15
2014-06-25
2014-07-05
2014-07-15
2014-07-25
2014-08-04
2014-08-14
2014-08-24
2014-09-03
2014-09-13
2014-09-23
2015
2015-04-13
2015-04-23
2015-05-03
2015-05-13
2015-05-23
2015-06-02
2015-06-12
2015-06-22
2015-07-02
2015-07-12
2015-07-22
2015-08-01
2015-08-11
2015-08-21
2015-08-31
2015-09-10
2015-09-20
2016
2016-04-10
2016-04-20
2016-04-30
2016-05-10
2016-05-20
2016-05-30
2016-06-09
2016-06-29
2016-07-09
2016-07-19
2016-07-29
2016-08-08
2016-08-18
2016-08-28
2016-09-07
2016-09-17
2016-09-27
2017
2017-04-07
2017-04-17
2017-04-27
2017-05-07
2017-05-17
2017-05-27
2017-06-06
2017-06-16
2017-06-26
2017-07-06
2017-07-16
2017-07-26
2017-08-05
2017-08-15
2017-08-25
2017-09-04
2017-09-14
2017-09-24
2018
2018-04-04
20

In [7]:
# define the unwanted columns
useless = ['Result',
 'Pitcher (W)',
 'Pitcher (L)',
 'Pitcher (S)',
 'Home_Hit_Date',
 'Home_Hit_Team',
 'Home_Hit_TG',
 'Away_Hit_Date',
 'Away_Hit_Team',
 'Away_Hit_TG',
 'Home_Pit_Date',
 'Home_Pit_Team',
 'Home_Pit_TG',
 'Away_Pit_Date',
 'Away_Pit_Team',
 'Away_Pit_TG']

In [8]:
# filter out unnecessary columns
cols = [x for x in list(pd.concat(all_stuff).columns) if x not in useless]

In [9]:
fin_df = pd.concat(all_stuff)[cols]

In [10]:
# account for differences between the cbs and fangraphs acronyms
col_convs = {'WSN':'WAS','KCR':'KC','SDP':'SD','SFG':'SF','TBR':'TB'}

In [11]:
fin_df.Home = fin_df.Home.replace(col_convs)
fin_df.Away = fin_df.Away.replace(col_convs)

In [12]:
fin_df['Target'] = fin_df.Home == fin_df.Winner

In [13]:
fin_df.shape

(21020, 158)

In [22]:
from pathlib import Path
downloads_path = str(Path.home() / "Downloads")

In [23]:
# save the data set as a pickle object
fin_df.to_pickle(downloads_path + '/data_w_correct_cols.pkl')

In [24]:
# save the data set as a csv file
fin_df.to_csv(downloads_path + '/data_w_correct_cols.csv')